In [1]:
import os                       # os tillader os bl.a. finder filplaceringer på computeren
import numpy as np              # Numpy leverer noget af matematikken, der ligger under Pandas 
import pandas as pd             # Pandas tillader os at importere, oprette og manipulere data frames
from pandas import DataFrame    # Nogle libraries har under-biblioteker. Underbiblioteker importeres med from-kommandoen
import matplotlib.pyplot as plt # så jeg kan lave grafer
from nltk.text import Text      # bruges til tekstanalyse
import glob                     # hjælper med paths og filnavne
import re

In [2]:
!python -m  spacy download da_core_news_lg

[+] Download and installation successful
You can now load the package via spacy.load('da_core_news_lg')


In [3]:
import spacy
nlp = spacy.load("da_core_news_lg")

In [4]:
sagn = [] # opretter tom liste

for fil in os.scandir(r'.\sagn'): # for-loop
    with open (fil, encoding = "utf8") as f: # context manager
        sagn.append(f.read().replace("\n"," ").replace("*"," ")) # tilføj renset tekst til liste

In [5]:
sagn

['Sagn: Bispehøjen  Oversat fra færøsk af Anker Eli Petersen © 2005  originaltekst: Biskupsheygur Færøske Folkesagn og Æventyr Jakob Jakobsen, København 1898-1901    Rasmus Ganting, en præst fra Vágoy, var engang på vej til Sørvág sammen med sin karl. Stien går langs indsøen Leitisvatn, mellem Midvág og Sørvág, og tæt forbi en høj kaldet Biskupsheygur, tæt ved søen.    Da de skulle passere højen, stod den åben, og en kvinde stod i døren og inviterede præsten indenfor. En anden kvinde kom med øl i et sølvhorn og bød ham at drikke, men han blæste skummet af det, lige i fjæset på hende, før han drak. Da sagde hun    Du er en vis mand. Men præsten svarede, at hvis han ikke var mere lærd end hun, så ville han ikke være kommet.    De blev i højen hele dagen, og om aftenen blev en seng stillet til deres rådighed. Karlen var meget bange og kunne ikke sove, men præsten bad ham holde fast i sin fod og rykke i den, hvis han sov over sig.    Troldene sloges indbyrdes og beskyldte hinanden for at h

In [6]:
def rens_ord(text_0):
    text_1 = text_0.replace("\n"," ")
    text_2 = text_1.replace("."," ")
    text_3 = text_2.replace(","," ")
    text_4 = text_3.replace(":"," ")
    text_5 = text_4.replace("*"," ")
    text_6 = text_5.replace("–"," ")
    text_7 = text_6.replace("'"," ")
    text_8 = text_7.replace("”"," ")
    text_ren = text_8.replace("-"," ")
    text_lav = text_ren.lower()
    text_token = text_lav.split()
    return text_token                      #renser ord i teksterne

In [7]:
def rens_sæt(text_0):
    text_1 = text_0.replace("\n"," ")
    text_2 = text_1.replace(","," ")
    text_3 = text_2.replace(":"," ")
    text_4 = text_3.replace("*"," ")
    text_5 = text_4.replace("–"," ")
    text_6 = text_5.replace("'"," ")
    text_7 = text_6.replace("”"," ")
    text_ren = text_7.replace("-"," ")
    text_lav = text_ren.lower()
    text_token = text_lav.split(".")
    return text_token                      #renser sætninger i teksterne

In [8]:
sagn

['Sagn: Bispehøjen  Oversat fra færøsk af Anker Eli Petersen © 2005  originaltekst: Biskupsheygur Færøske Folkesagn og Æventyr Jakob Jakobsen, København 1898-1901    Rasmus Ganting, en præst fra Vágoy, var engang på vej til Sørvág sammen med sin karl. Stien går langs indsøen Leitisvatn, mellem Midvág og Sørvág, og tæt forbi en høj kaldet Biskupsheygur, tæt ved søen.    Da de skulle passere højen, stod den åben, og en kvinde stod i døren og inviterede præsten indenfor. En anden kvinde kom med øl i et sølvhorn og bød ham at drikke, men han blæste skummet af det, lige i fjæset på hende, før han drak. Da sagde hun    Du er en vis mand. Men præsten svarede, at hvis han ikke var mere lærd end hun, så ville han ikke være kommet.    De blev i højen hele dagen, og om aftenen blev en seng stillet til deres rådighed. Karlen var meget bange og kunne ikke sove, men præsten bad ham holde fast i sin fod og rykke i den, hvis han sov over sig.    Troldene sloges indbyrdes og beskyldte hinanden for at h

In [9]:
titler = ["Bispehøjen", "Bryllup i Stapi", "Dværge", "Femte, Sakaris og strandtroldene i Hattarvík", "Havfrue", "Havkvæg og huldekvæg", "Huldebarnet", "Huldekvindens barsel", "Huldemanden i Guldstenen", "Huldemanden og fårehyrden", "Huldemanden under liðastein", "Klog Marjun og vætterne", "Manden i huldebåden", "Maren", "Marmennill", "Niðagrísur i gamachen", "Niðagrísur og præsten i Ónagerði", "Niðagrísur", "Nøkken fra Eiði", "Nøkken i Leitisvatn", "Nøkken", "Nøkken2", "Pálin undir Hamri", "Pukrhuset", "Símun í Kirkjubø", "Sælkvinden fra Skálavík", "Sælkvinden", "Títil-Tátá", "Troldene på Skálavøllum", "Troldenæs", "Vætterne i Skáli"] 

In [10]:
df_sagn = pd.DataFrame ({"Titler": titler, "Sagn": sagn,})

In [11]:
df_sagn

,Titler,Sagn
0,Bispehøjen,Sagn: Bispehøjen Oversat fra færøsk af Anker ...
1,Bryllup i Stapi,Sagn: Brylluppet i Stapi Oversat fra færøsk a...
2,Dværge,Sagn: Dværge Oversat fra færøsk af Anker Eli ...
3,"Femte, Sakaris og strandtroldene i Hattarvík","Sagn: Femte, Sakaris og strandtroldene i Hatta..."
4,Havfrue,Sagn: Havfrue Oversat fra færøsk af Anker Eli...
5,Havkvæg og huldekvæg,Sagn: Havkvæg og huldekvæg Oversat fra færøsk...
6,Huldebarnet,Sagn: Huldebarnet (Skiftingen) Oversat fra fæ...
7,Huldekvindens barsel,Sagn: Huldekvindens barsel Oversat fra færøsk...
8,Huldemanden i Guldstenen,Sagn: Huldemanden i Guldstenen Oversat fra fæ...
9,Huldemanden og fårehyrden,Sagn: Huldemanden og fårehyrden Oversat fra f...


In [12]:
df_sagn["renTxtOrd"] = df_sagn.Sagn.apply(rens_ord)

In [13]:
df_sagn["antalOrd"] = df_sagn.renTxtOrd.apply(len)

In [14]:
df_sagn

,Titler,Sagn,renTxtOrd,antalOrd
0,Bispehøjen,Sagn: Bispehøjen Oversat fra færøsk af Anker ...,"[sagn, bispehøjen, oversat, fra, færøsk, af, a...",279
1,Bryllup i Stapi,Sagn: Brylluppet i Stapi Oversat fra færøsk a...,"[sagn, brylluppet, i, stapi, oversat, fra, fær...",1207
2,Dværge,Sagn: Dværge Oversat fra færøsk af Anker Eli ...,"[sagn, dværge, oversat, fra, færøsk, af, anker...",296
3,"Femte, Sakaris og strandtroldene i Hattarvík","Sagn: Femte, Sakaris og strandtroldene i Hatta...","[sagn, femte, sakaris, og, strandtroldene, i, ...",743
4,Havfrue,Sagn: Havfrue Oversat fra færøsk af Anker Eli...,"[sagn, havfrue, oversat, fra, færøsk, af, anke...",138
5,Havkvæg og huldekvæg,Sagn: Havkvæg og huldekvæg Oversat fra færøsk...,"[sagn, havkvæg, og, huldekvæg, oversat, fra, f...",216
6,Huldebarnet,Sagn: Huldebarnet (Skiftingen) Oversat fra fæ...,"[sagn, huldebarnet, (skiftingen), oversat, fra...",468
7,Huldekvindens barsel,Sagn: Huldekvindens barsel Oversat fra færøsk...,"[sagn, huldekvindens, barsel, oversat, fra, fæ...",274
8,Huldemanden i Guldstenen,Sagn: Huldemanden i Guldstenen Oversat fra fæ...,"[sagn, huldemanden, i, guldstenen, oversat, fr...",222
9,Huldemanden og fårehyrden,Sagn: Huldemanden og fårehyrden Oversat fra f...,"[sagn, huldemanden, og, fårehyrden, oversat, f...",285


In [15]:
df_sagn["renTxtSæt"] = [rens_sæt(txt) for txt in df_sagn["Sagn"]]

In [16]:
df_sagn["antalSæt"] = [len(txt) for txt in df_sagn["renTxtSæt"]]

In [17]:
df_sagn["antalTegn"] = [len(txt) for txt in df_sagn["Sagn"]]

In [18]:
df_sagn

,Titler,Sagn,renTxtOrd,antalOrd,renTxtSæt,antalSæt,antalTegn
0,Bispehøjen,Sagn: Bispehøjen Oversat fra færøsk af Anker ...,"[sagn, bispehøjen, oversat, fra, færøsk, af, a...",279,[sagn bispehøjen oversat fra færøsk af anker...,16,1536
1,Bryllup i Stapi,Sagn: Brylluppet i Stapi Oversat fra færøsk a...,"[sagn, brylluppet, i, stapi, oversat, fra, fær...",1207,[sagn brylluppet i stapi oversat fra færøsk ...,71,6565
2,Dværge,Sagn: Dværge Oversat fra færøsk af Anker Eli ...,"[sagn, dværge, oversat, fra, færøsk, af, anker...",296,[sagn dværge oversat fra færøsk af anker eli...,23,1587
3,"Femte, Sakaris og strandtroldene i Hattarvík","Sagn: Femte, Sakaris og strandtroldene i Hatta...","[sagn, femte, sakaris, og, strandtroldene, i, ...",743,[sagn femte sakaris og strandtroldene i hatt...,38,4054
4,Havfrue,Sagn: Havfrue Oversat fra færøsk af Anker Eli...,"[sagn, havfrue, oversat, fra, færøsk, af, anke...",138,[sagn havfrue oversat fra færøsk af anker el...,9,786
5,Havkvæg og huldekvæg,Sagn: Havkvæg og huldekvæg Oversat fra færøsk...,"[sagn, havkvæg, og, huldekvæg, oversat, fra, f...",216,[sagn havkvæg og huldekvæg oversat fra færøs...,15,1277
6,Huldebarnet,Sagn: Huldebarnet (Skiftingen) Oversat fra fæ...,"[sagn, huldebarnet, (skiftingen), oversat, fra...",468,[sagn huldebarnet (skiftingen) oversat fra f...,28,2650
7,Huldekvindens barsel,Sagn: Huldekvindens barsel Oversat fra færøsk...,"[sagn, huldekvindens, barsel, oversat, fra, fæ...",274,[sagn huldekvindens barsel oversat fra færøs...,16,1508
8,Huldemanden i Guldstenen,Sagn: Huldemanden i Guldstenen Oversat fra fæ...,"[sagn, huldemanden, i, guldstenen, oversat, fr...",222,[sagn huldemanden i guldstenen oversat fra f...,11,1250
9,Huldemanden og fårehyrden,Sagn: Huldemanden og fårehyrden Oversat fra f...,"[sagn, huldemanden, og, fårehyrden, oversat, f...",285,[sagn huldemanden og fårehyrden oversat fra ...,14,1591


In [19]:
df_sagn.describe()

,antalOrd,antalSæt,antalTegn
count,31.000000,31.000000,31.000000
mean,393.612903,21.903226,2183.161290
std,335.708969,17.675510,1818.277502
min,138.000000,7.000000,781.000000
25%,221.500000,12.000000,1240.000000
50%,296.000000,16.000000,1591.000000
75%,399.000000,23.000000,2329.000000
max,1690.000000,85.000000,9202.000000


In [20]:
df_sagn["gnmSætLæng"] =df_sagn["antalOrd"]/df_sagn["antalSæt"]

In [21]:
df_sagn.to_csv("sagn_datasæt_renset.csv", index=False)

In [22]:
df_sagn.to_csv

<bound method NDFrame.to_csv of                                           Titler  \
0                                     Bispehøjen   
1                                Bryllup i Stapi   
2                                         Dværge   
3   Femte, Sakaris og strandtroldene i Hattarvík   
4                                        Havfrue   
5                           Havkvæg og huldekvæg   
6                                    Huldebarnet   
7                           Huldekvindens barsel   
8                       Huldemanden i Guldstenen   
9                      Huldemanden og fårehyrden   
10                   Huldemanden under liðastein   
11                       Klog Marjun og vætterne   
12                           Manden i huldebåden   
13                                         Maren   
14                                    Marmennill   
15                         Niðagrísur i gamachen   
16              Niðagrísur og præsten i Ónagerði   
17                              

In [63]:
def find_navn(renTxtSæt,navn):
    antal = txt.count(navn)
    return antal

In [64]:
items1 = ("havfrue", "havfruen", "havfruerne", "sirene", "sirenen", "sirenerne", "havmand", "havmænd", "havmanden", "havmændene", "nøkken")

In [65]:
items2 = ("dværg", "dværge", "dværgene", "dværgen") 

In [66]:
items3 = ("hulde", "hulden", "huldekvinde", "huldekvinden", "huldebarn", "huldebarnet", "huldemand", "huldemanden", "huldefolk",  "huldefolket", "huldre", "huldrerne")

In [67]:
items4 = ("trolde", "trolden", "Troldene", "troldekone")

In [68]:
from collections import Counter

In [69]:
counts = Counter(items1)

In [72]:
counts1 = Counter(items4)

In [76]:
Trold=counts1

In [78]:
Trold

Counter({'trolde': 1, 'trolden': 1, 'Troldene': 1, 'troldekone': 1})

In [70]:
havfrue = counts["havfrue"]

In [71]:
havfrue

1

In [46]:
havfruen = counts ["havfruen"]

In [47]:
havfruen

0

In [48]:
sagn[0]

'Sagn: Bispehøjen  Oversat fra færøsk af Anker Eli Petersen © 2005  originaltekst: Biskupsheygur Færøske Folkesagn og Æventyr Jakob Jakobsen, København 1898-1901    Rasmus Ganting, en præst fra Vágoy, var engang på vej til Sørvág sammen med sin karl. Stien går langs indsøen Leitisvatn, mellem Midvág og Sørvág, og tæt forbi en høj kaldet Biskupsheygur, tæt ved søen.    Da de skulle passere højen, stod den åben, og en kvinde stod i døren og inviterede præsten indenfor. En anden kvinde kom med øl i et sølvhorn og bød ham at drikke, men han blæste skummet af det, lige i fjæset på hende, før han drak. Da sagde hun    Du er en vis mand. Men præsten svarede, at hvis han ikke var mere lærd end hun, så ville han ikke være kommet.    De blev i højen hele dagen, og om aftenen blev en seng stillet til deres rådighed. Karlen var meget bange og kunne ikke sove, men præsten bad ham holde fast i sin fod og rykke i den, hvis han sov over sig.    Troldene sloges indbyrdes og beskyldte hinanden for at ha

In [49]:
trold = counts ["trolde", "trolden", "Troldene", "troldekone"]

In [50]:
trold

0